In [1]:
# Lecture by IBM Qiskit & Jin-Sung Kim, Phd.
# Notes & Solution by Akhona Njeje.
# Date : 26 Dec 2023.
# Topic : Quantumn Physics Algorithms.

# Theory :
# 1 of the known ways a QC can outperform a CC(Classical Computer) is Grovers Algotihm.
# What is a QA(Quantum Algotithm)?its an algorithm that takes advantage of Superposition & Entanglement to perform computation at a complex level.
# Linear Regression(LR) is not a QA & is run on CC & CC runs on binary logic.

In [2]:
list = [1, 3, 5, 2, 2, 9, 5, 8, 0, 7, 6] # List of random numbers, we want to find the element 7. 7 is #9 on the list.
                                         # We can employ an Oracle used in QC to tell us if the number 7, y/n.
                                         # Oracle is like our black box.
def oracle(my_input):
    winner = 7
    if my_input is winner:
        response = True
    else:
        response = False
    return response

In [3]:
# Lets check how many times must we call the oracle for us to find the winner.
for index , trial_number in enumerate(list):
    if oracle(trial_number) is True:
        print('Winner found at index %i'%index)
        print('%i calls to the Oracle used'%(index+1))
        break
        
# CC has an Algorithm O(N). We must call the oracle n number of times to find the winner.
# QC has an Algorithm O(N^0.5) = Grovers Algorithm.
# To solve this problem we used Binary operations used in CC, lets turn to QC.

Winner found at index 9
10 calls to the Oracle used


In [7]:
# To solve this problem using QC, we need Q#.

import os
os.chdir(r'C:\Users\User\Desktop\Ak Projects\Mathematical Physics\Quantum Physics App in QC\Quantum Algorithms\Q')
from qiskit import * # for qiskit launch the Anaconda Powershell Prompt to use the frame work.
import matplotlib.pyplot as plt 
import numpy as np

In [8]:
# Lets define the oracle circuit.

oracle = QuantumCircuit(2, name='oracle')
oracle.cz(0, 1)
oracle.to_gate()
oracle.draw()

# lets let our winner be 11, when we feed 11 to the black box we must get -11, thats how a blackbox works.
# To compute these states we will need to use Unitary matrices.
# To convert [11> to -[11> we will use Controlled Z Gate.

q_0: ─■─
      │ 
q_1: ─■─

In [22]:
# Lets prepare a super position state, its gona help us check if our circuit is doing what we expect.
# We will need H Gates to acheive this.
# Lets call this state S for super position.

from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.providers import aer

backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2, 2)   # 2 QUBITS & 2 CLASSICAL REGISTORS.
grover_circ.h([0, 1])   # adding the H gates on both Qubits 0 & 1.
grover_circ.append(oracle, [0, 1])   # We query two Qubits at the same time.
grover_circ.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [23]:
# Execute this job on the simulator.

job = execute(grover_circ, backend)
result = job.result()

In [24]:
# Lets get back the state vector.

sv = result.get_statevector()
np.around(sv, 2)   # round of to two decimals.

# We get the results of the 11 state.
# But we are not done, in Q.Physics we have to square the state vector in oder to get back the probabilities.
# This eliminates the -(minus) sign on -[11>.

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [26]:
# We have to amplify the probabilities of winning states vs non-winning by reducing them.
# [w> = [0 0 0 1] = [11> = winning state = 1 column.
# [s> = [1 1 1 1]*0.5 = super position = 1 column.
# [s'> = [1 1 1 0]*1/sqrt(3).
# [w> * [s'> = -1, meaning [w> & [s'> are perpendicular.

reflection = QuantumCircuit(2, name='reflection')
reflection.h([0, 1])
reflection.z([0, 1])
reflection.cz(0, 1)
reflection.h([0, 1])
reflection.to_gate()
reflection.draw()   # Lets check how the reflection operator looks.

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [27]:
# Lets put everything together.

backend = Aer.get_backend('qasm_simulator')
grover_circ = QuantumCircuit(2, 2)
grover_circ.h([0, 1])
grover_circ.append(oracle, [0, 1])
grover_circ.append(reflection, [0, 1])
grover_circ.measure([0, 1], [0, 1])
grover_circ.draw()   # We have prepared our super position state. At the end we want the state [11>, execute the job.

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [29]:
job = execute(grover_circ, backend, shots=1)
result = job.result()
result.get_counts() # {'11', 1} = {'winning state', 1 call to the oracle}
# Using 1 call to the oracle we used Grovers Algorithm to acheive this.

{'11': 1}